### LLM Application with LCEL

1. Langchain expression language - Chain components together
2. Language translator

In [1]:
# Open AI Key and Open source models - Llama3, Gemma2, mistral using GROQ

import os
from dotenv import load_dotenv

load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [2]:
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014A735A4940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A735A5CF0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

In [4]:
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, How are you?")
]

result = model.invoke(messages)

In [5]:
result

AIMessage(content='**Bonjour, comment allez-vous ?** \n\n\nThis is the most formal way to say "Hello, how are you?" in French.  \n\nHere are some other options:\n\n* **Salut, ça va ?** (Informal, more like "Hi, how\'s it going?")\n* **Coucou, comment vas-tu ?** (Very informal, like "Hey, how are you?") \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 22, 'total_tokens': 129, 'completion_time': 0.194545455, 'prompt_time': 0.002147966, 'queue_time': 0.233762431, 'total_time': 0.196693421}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ae86c12-6cda-4b50-a483-5e8b230d712c-0', usage_metadata={'input_tokens': 22, 'output_tokens': 107, 'total_tokens': 129})

In [6]:
result.content

'**Bonjour, comment allez-vous ?** \n\n\nThis is the most formal way to say "Hello, how are you?" in French.  \n\nHere are some other options:\n\n* **Salut, ça va ?** (Informal, more like "Hi, how\'s it going?")\n* **Coucou, comment vas-tu ?** (Very informal, like "Hey, how are you?") \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n'

In [7]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

parsed_result = parser.invoke(result)
parsed_result

'**Bonjour, comment allez-vous ?** \n\n\nThis is the most formal way to say "Hello, how are you?" in French.  \n\nHere are some other options:\n\n* **Salut, ça va ?** (Informal, more like "Hi, how\'s it going?")\n* **Coucou, comment vas-tu ?** (Very informal, like "Hey, how are you?") \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n'

In [10]:
# Using LCEL, we can change the components

chain = model | parser

chain.invoke(messages)

'Here are a few ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, singular)\n* **Salut, comment allez-vous ?** (Informal, plural)\n\n\nThe best option depends on the context and who you are speaking to. \n'

In [11]:
# Prompt templates

from langchain_core.prompts import ChatPromptTemplate

In [12]:
generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [14]:
result = prompt.invoke(
    {"language": "French", "text": "Hello"}
)

In [15]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [16]:
chain = prompt | model | parser

In [17]:
chain.invoke({"language": "French", "text": "Hello"})

'Bonjour \n'